In [ ]:
!git clone https://github.com/MarkBridge11/RINE-extended

In [ ]:
import sys
sys.path.append('/content/RINE-extended')

In [ ]:
!pip install scikit-image imageio kornia

In [ ]:
import sys
import os
from src.utils import get_transforms, get_our_trained_model
from PIL import Image
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_, transforms, _ = get_transforms()
model = get_our_trained_model(ncls="ldm", device=device)
model.to(device)
model.eval()

In [ ]:
real_path = "demo/aadlygmazf.jpg"
real_image = Image.open(real_path).convert("RGB")
real_tensor = transforms(real_image).unsqueeze(0).to(device)
real_logit = model(real_tensor)[0]
real_probability = torch.sigmoid(real_logit)
print(
    f"real image - prob. to be fake: {real_probability.detach().cpu().numpy()[0][0]*100:1.1f}%"
)

fake_path = "demo/zzrybysskm.jpg"
fake_image = Image.open(fake_path).convert("RGB")
fake_tensor = transforms(fake_image).unsqueeze(0).to(device)
fake_logit = model(fake_tensor)[0]
fake_probability = torch.sigmoid(fake_logit)
print(
    f"fake image - prob. to be fake: {fake_probability.detach().cpu().numpy()[0][0]*100:1.1f}%"
)
